# Todo
* better comments
* once through to clean up
* 2004, 2005, 2006 are commented out due to quality issues. mostly around lea_type and aun

In [1]:
import ssl
import re
import pandas as pd
import numpy as np
import urllib.parse
import os
import requests
import sqlite3 as db

# gets rid of ssl errors
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re

# set this to the list of expected columns
int_columns = ['PKA', 'PKP', 'PKF', 'K4A', 'K4P', 'K4F', 'K5A', 'K5P', 'K5F',
                 '001', '002', '003', '004', '005', '006', '007', '008', '009',
                 '010', '011', '012', 'school_year']
string_columns = ['LEA Name', 'LEA Type', 'County', 'School Name', 'AUN', 'School Number']
expected_columns = string_columns + int_columns

url = "https://www.education.pa.gov/DataAndReporting/Enrollment/Pages/PublicSchEnrReports.aspx"
host = re.sub(r'(https*://[^/]+).*','\\1',url)
req = Request(url)
html_page = urlopen(req)

soup = BeautifulSoup(html_page, "html")

links = []
for link in soup.findAll('a'):
    links.append(link.get('href'))

In [3]:
# this filters down the list to just what we are looking for
links = list( filter(lambda l: re.search(r'Enrollment%20Public%20Schools*%2020\d\d-\d\d\.xlsx*$', str(l) ) , links))

In [4]:
# this will put the host part of the URL back on the beginning of the link
links = list(map( lambda l: host + l, links))

In [5]:
# this function will download a file if local cache doesn't exist and then return local file name
def cache_url(file):

    # set directory base and create if doesn't exist
    directory='../data/raw/basic_enrollment/'
    try:
        os.stat(directory)
    except:
        os.makedirs(directory)

    # decode the URL name. then split on /. and grab last item in list which will be the file name
    dec=urllib.parse.unquote( file )
    file_list=dec.split('/')
    len_list=len(file_list)
    cache_file=directory + file_list[len_list-1].lower().replace(' ', '_')

    # see if file exists. if so use that. otherwise download it
    if not os.path.exists( cache_file ):
        print( 'caching file: ' + cache_file )
        file_to_write = requests.get(file)
        with open(cache_file, 'wb') as f:
            f.write(file_to_write.content)
        
    return cache_file

# this will do the download, just pass it a sheet and URL or file name
def download_excel(file,sheet,skiprows):
        # download and read in the sheet
        df = pd.DataFrame()

        # download, cache file, and return cached name
        file=cache_url(file)

        try:
            df = pd.read_excel(file,
                       sheet_name=sheet,
                       skiprows=skiprows)
        except:
            print( 'Error: not able to download: ' + file )
            
        return df

In [6]:
# builds one giant dataframe
df = pd.DataFrame()
count=0
for l in links: 
    if count < 120:
        # find the start year from the URL
        school_year = re.sub('^https.+(\d\d\d\d)-\d\d.xlsx*$','\\1',l)
        #print( school_year + ": " + l )

        # download and read in the sheet
        if int(school_year) >= 2011:
            df1 = download_excel(l,'LEA and School', 4)
        elif int(school_year) == 2010:
            df1 = download_excel(l,'School', 4)
        elif int(school_year) == 2009:
            df1 = download_excel(l,'School - Data File', 4)
        elif int(school_year) == 2008:
            df1 = download_excel(l,'School - Datafile', 4)
        elif int(school_year) == 2007:
            df1 = download_excel(l,'LEA - Data File', 6)
        # these are commented out as more work is required to clean up the data
        #elif int(school_year) == 2006:
        #    df1 = download_excel(l,'School Enrollments', 1)
        #elif int(school_year) == 2005:
        #    df1 = download_excel(l,'School Enrollments by LEA', 2)
        #elif int(school_year) == 2004:
        #    df1 = download_excel(l,'School Enrollments', 2)


        # 2006 has some bad columns in it. this makes them conform
        col = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
        for c in col:
            if c in df1.columns:
                nc = "00" + str(c)
                nc = nc[-3:]
                df1.rename(columns={c: nc}, inplace=True)
        
        # more 2006. They don't break K into AM, PM, Full. So put all numbers in AM
        if 'PreK' in df1.columns:
            df1.rename(columns={"PreK": "PKA"}, inplace=True)
        if 'K4' in df1.columns:
            df1.rename(columns={"K4": "K4A"}, inplace=True)
        if 'K5' in df1.columns:
            df1.rename(columns={"K5": "K5A"}, inplace=True)
        
        # 2006 uses county name instead of county
        if "County Name" in df1.columns:
            df1.rename(columns={"County Name": "County"}, inplace=True)
        
        # 2011 has a 7 as a column name. renaming that. its an int 7 not a string 7. 
        if 7 in df1.columns:
            df1.rename(columns={7: "007"}, inplace=True)

        # add LEA Type by using the end of the LEA Name. For example Avon Grove School
        # district would be "Avon Grove SD". So pulling off the right most string without 
        # a space will get me the LEA Type
        if 'LEA Type' not in df1.columns:
            pd.Series(df1['LEA Name'], dtype="string") # doing this now because in 2005 its a strange data type
            df1['LEA Type'] = df['LEA Name'].apply(lambda x: re.sub(' (\S+)$','\\1',x))       

        # add in school year
        df1['school_year'] = school_year
         
        # get rid of any row where school number isn't a number
        if 'School Number' in df1.columns:
            df1['School Number'] = pd.Series(df1['School Number'], dtype="string")
            df1 = df1[df1['School Number'].str.contains('^[\d\.]+$', regex= True, na=False)]
        
        # fix datatypes
        for col in int_columns:
            # if doesn't exist create it
            if col not in df1.columns:
                df1[col] = np.nan
            df1[col] = pd.Series(df1[col], dtype="UInt16")   
                
        for col in string_columns:
            # if doesn't exist create it
            if col not in df1.columns:
                df1[col] = np.nan
            df1[col] = pd.Series(df1[col], dtype="string") 

        # some of the older files have pivot table looking reports. Where say the 
        # county is in row 1 with 200 schools, and the next 199 rows they don't
        # repeat the county. So this fills in County and the School District or 
        # LEA Name
        fields = ['County', 'LEA Name', 'AUN']
        for f in fields:
            field = ''
            for i, row in df1.iterrows():
                if not pd.isna(df1.at[i,f]):
                    field = df1.at[i,f]
                else:
                    df1.at[i,f] = field
            
        # this drops un-used columns. compares retrieved col to expected ones
        for col in df1.columns.difference(expected_columns):
            if col in df1.columns:
                df1.drop(columns=[col], inplace=True)

        # drops any rows where LEA Name ends in Total
        df1 = df1[~df1['LEA Name'].str.contains('^.*Total$', regex= True, na=False)]
        
        # concat the newly downloaded df onto the larger one
        df = pd.concat([df, df1],
                       ignore_index=True,)
    count = count + 1

# convert header names to strings
df.columns = df.columns.map(str)

In [7]:
# Pick better names
df.columns = [column.strip().replace(' ', '_').lower() for column in df.columns]

In [8]:
## fill in blank aun. years 2006 and earlier don't have aun. So by lea_name grab first aun going
## back in time. and use that to fill in the blanks

# keep the first aun we find for each SD
df_aun = df.drop_duplicates(subset='lea_name', keep="first")[['aun', 'lea_name']]

# rename aun column
df_aun.rename(columns={"aun": "aun_fix"}, inplace=True)

# join on lea_name
df = df.join(df_aun.set_index('lea_name'), on='lea_name')

# only if aun is empty put in the joined aun_fix
df['aun'] = df.apply(lambda x: x['aun_fix'] if x['aun']=='' else x['aun'], axis=1)

# drop fix columns
df.drop(columns=['aun_fix'], inplace=True)

In [9]:
# melt to long format
df = pd.melt(df, id_vars=['aun', 'lea_name', 'lea_type', 'county', 'school_number', 'school_name', 'school_year'],
        var_name='grade', value_name='enrollment')

In [10]:
# to make more efficient change data type on columns with lots of repeating values to category
for col in df.columns:
    if col != 'enrollment':
        df[col] = pd.Series(df[col], dtype="category") 
    else:
        df[col] = pd.Series(df[col], dtype="UInt16")   

In [11]:
# save the data out for another script to consume
df.to_pickle("../data/basic_enrollment.pkl.bz2", compression='infer')

## Below this will be putting summarized tables into sqlite3 database

In [12]:
# Connect to DB file
cnx = db.connect('../data/pde.db')

In [13]:
# create lea_info table -> using most recent information about aun so to normalize throughout years
desired_columns=['aun', 'lea_name', 'lea_type', 'county']                     # list of columns we want
lea_info = df.sort_values(by = 'school_year', ascending = False)  # sort by school_year newest first. will grab most recent entry
lea_info = lea_info[desired_columns]                              # throw out columns we don't want
lea_info = lea_info.drop_duplicates(subset='aun', keep='first', inplace=False, ignore_index=True)
lea_info = lea_info.set_index(['aun'])
lea_info.to_sql(name='lea_info', con=cnx, if_exists='replace', index=True)

In [14]:
# create school_into table -> using most recent information about school so to normalize throughout years
desired_columns=['school_number', 'school_name', 'aun']              # list of columns we want
school_info = df.sort_values(by = 'school_year', ascending = False)  # sort by school_year newest first. will grab most recent entry
school_info = school_info[desired_columns]                           # throw out columns we don't want
school_info = school_info.drop_duplicates(subset='school_number', keep='first', inplace=False, ignore_index=True)
school_info = school_info.set_index(['school_number'])
school_info.to_sql(name='school_info', con=cnx, if_exists='replace', index=True)

In [15]:
# create basic_enrollment table
desired_columns=['aun', 'school_number', 'school_year', 'grade', 'enrollment']    # list of columns we want
basic_enrollment = df[desired_columns]                                            # throw out columns we don't want
basic_enrollment = basic_enrollment.set_index(['aun', 'school_number', 'school_year', 'grade'])
basic_enrollment.to_sql(name='basic_enrollment', con=cnx, if_exists='replace', index=True)

In [16]:
# create summarized lea enrollment
lea_w_enrollment = pd.pivot_table(df,index=["aun", "school_year"], values=["enrollment"],aggfunc=np.sum, observed=True)
lea_w_enrollment.to_sql(name='lea_w_enrollment', con=cnx, if_exists='replace', index=True)

In [17]:
# create summarized school enrollment
school_w_enrollment = pd.pivot_table(df,index=["school_number", "school_year"], values=["enrollment"],aggfunc=np.sum, observed=True)
school_w_enrollment.to_sql(name='school_w_enrollment', con=cnx, if_exists='replace', index=True)

In [18]:
# close the database
cnx.close()